#### imports

In [1]:
import math
import lucene
import time
import nltk
import itertools
import numpy as np
from tqdm import tqdm
from java.io import File
import xml.etree.ElementTree as ET
from collections import defaultdict
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.util import BytesRefIterator
from org.apache.lucene.index import DirectoryReader, Term
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.analysis.core import WhitespaceAnalyzer
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.search import IndexSearcher, BooleanQuery, BooleanClause, TermQuery, BoostQuery
from org.apache.lucene.search.similarities import BM25Similarity, LMJelinekMercerSimilarity, LMDirichletSimilarity
lucene.initVM()

In [2]:
q_name = 'trec678-robust'

In [3]:
index_path = '../../../index/'
topicFilePath = f'../../../{q_name}.xml'
qrel_file = '../../../trec678_robust.qrel'

directory = FSDirectory.open(File(index_path).toPath())
indexReader = DirectoryReader.open(directory)

In [4]:
def query_topics(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    parsed_topics = {}

    for top in root.findall('top'):
        num = top.find('num').text.strip()
        title = top.find('title').text.strip()
        parsed_topics[num] = title

    return parsed_topics

In [5]:
def makeRelJudgeDict(qrelFilePath):
    # {qid1:{docid1:0/1,docid2:0/1,...}, qid2:{docid2:0/1,docid4:0/1,...},...}
    relJudgeDict = {}
    with open(qrelFilePath, 'r') as f:
        for line in f:
            l = line.split()
            qid, docid, judgement = l[0], l[2], int(l[3])
            if qid not in relJudgeDict:
                relJudgeDict[qid] = {docid: judgement}
            else:
                relJudgeDict[qid][docid] = judgement
    return relJudgeDict

def isTrueRelevant(qid, docid, relJudgeDict):
    # returns if the doc is True relevant, for the given query, according to the judgment file
    if qid not in relJudgeDict:
        return False
    if docid not in relJudgeDict[qid]:
        return False
    if relJudgeDict[qid][docid] == 1:   # 1 -> Relevant TRF
        return True
    if relJudgeDict[qid][docid] == 0:
        return False

def isTrueNonRelevant(qid, docid, relJudgeDict):
    # returns if the doc is NOT true relevant, for the given query, according to the judgment file
    if qid not in relJudgeDict:
        return False
    if docid not in relJudgeDict[qid]:
        return False
    if relJudgeDict[qid][docid] == 0:   # 0 -> Non-relevant TRF
        return True
    if relJudgeDict[qid][docid] == 1:
        return False

relJudgeDict = makeRelJudgeDict(qrel_file)

In [6]:
query_all = query_topics(topicFilePath)

In [7]:
def getDocumentVector(luceneDocid, indexReader):

    N = indexReader.numDocs()  
    # N = indexReader.getSumTotalTermFreq("CONTENTS")
                     
    
    docVec = defaultdict(lambda: [0, 0]) 
    D = 0                                 
    
    terms = indexReader.getTermVector(luceneDocid, 'CONTENTS')
    iterator = terms.iterator()
    for term in BytesRefIterator.cast_(iterator):
        t = term.utf8ToString()
        tf = iterator.totalTermFreq()  
        df = indexReader.docFreq(Term('CONTENTS', t))
        # cf = indexReader.totalTermFreq(Term('CONTENTS', t))
        D += tf
        docVec[t][0] = tf
        docVec[t][1] = df
    
    docVec = {key: (value[0] / D) * math.log(N / (value[1] + 1)) for key, value in docVec.items()}
    
    total_weight = sum(docVec.values())
    docVec = {key: value / total_weight for key, value in docVec.items()}

    return docVec


### RM3


In [8]:
def search(indexReader, query, similarity, top_rel_doc, qid, tpd, tnd):
    analyzer = EnglishAnalyzer()
    searcher = IndexSearcher(indexReader)
    searcher.setSimilarity(similarity)
    # query = QueryParser('CONTENTS', analyzer).escape(query)      # a few titles had '/' in them which 
    
    # query = QueryParser("CONTENTS", analyzer).parse(query)

    scoreDocs = searcher.search(query, top_rel_doc).scoreDocs
    
    docids = [scoreDoc.doc for scoreDoc in scoreDocs]

    relevent_docs = []
    nonrel_docs = []
    for id in docids:
        doc = searcher.doc(id)
        if isTrueRelevant(qid, doc.get('ID'), relJudgeDict):
            relevent_docs.append(id)
        if isTrueNonRelevant(qid, doc.get('ID'), relJudgeDict):
            nonrel_docs.append(id)
   
    # print(qid,relevent_docs)
    docids = relevent_docs[:tpd]
    ndocid = nonrel_docs[:tnd]
    
    set_cont = {term for doc in docids for term in getDocumentVector(doc, indexReader).keys()}

    
    
    # set_n = {term for doc in ndocid for term in getDocumentVector(doc, indexReader).keys()}
    
    # query_terms_set = set([term.strip()[9:] for term in query.toString().split()])
    # set_cont = set_cont.difference(set_n) | query_terms_set

    # set_cont = {ele for ele in set_cont if ele.isalpha()} 

    return set_cont, docids, ndocid

In [9]:
def RM3_term_selection(Query, set_ET, docs, ndocid, indexReader, alpha, lamb, expanded_query_terms):
    
    totalTF = indexReader.getSumTotalTermFreq("CONTENTS")

    Q = Query.split()
    weight = {}

    cf = {}
    for t in set_ET | set(Q):
        T = Term("CONTENTS", t)
        cf[t] = indexReader.totalTermFreq(T)/totalTF

    docVectors = {}
    ndocVectors = {}
   
    
    for d in docs:                    
        docVectors[d] = getDocumentVector(d, indexReader)
    
    for d in ndocid:                    
        ndocVectors[d] = getDocumentVector(d, indexReader)
        
        
    # tagged = nltk.tag.pos_tag(list(set_ET), tagset='universal')
    # set_ET = set([ele[0] for ele in tagged if ele[1] in ['NOUN','ADJ','ADP','X']])
    ml = lamb
    for w in set_ET:
        p_wr = 0
        tdc = 0
        for d in docs:                  
            tdc = tdc + 1
            # p_wd = (ml*(docVectors[d].get(w,0)) + (1 - ml)*cf[w]) 
            p_wd = docVectors[d].get(w,0)     
        
            p_q = 1
            for q in Q:
                # p_q = p_q*docVectors[d].get(q,0)   
                p_q = p_q*(ml*(docVectors[d].get(q,0)) + (1 - ml)*cf[q])   

            p_wr = p_wr + p_wd
        
        p_wr = p_wr/tdc

        p_wnr = 0
        ndc = 0 
        for d in ndocid:
            ndc = ndc + 1
            
            # p_wnd = (ml*(ndocVectors[d].get(w,0)) + (1 - ml)*cf[w]) 
            p_wnd = ndocVectors[d].get(w,0)

            p_q = 1
            for q in Q:
                # p_q = p_q*docVectors[d].get(q,0)   
                p_q = p_q*(ml*(ndocVectors[d].get(q,0)) + (1 - ml)*cf[q])

            p_wnr = p_wnr + p_wnd
        if ndc != 0:
            p_wnr = p_wnr/ndc
        
        p_wr = p_wr - p_wnr

        if p_wr > 0:
            weight[w] = p_wr


    weight = dict(sorted(weight.items(), key=lambda x:x[1], reverse=True)[:expanded_query_terms])
    
    norm = sum(weight.values())
    weight = {w:weight[w]/norm for w in weight}
 
    for w in weight.keys() | set(Q):
        weight[w] = (alpha*weight.get(w,0)) + (1-alpha)*(Q.count(w)/len(Q))
  

    temp_list = sorted(weight.items(), key=lambda x:x[1], reverse=True)
    sorted_weights = dict(temp_list)

    return sorted_weights

In [10]:
def expanded_query_BM25(search, RM3_term_selection, k1, b, alpha, top_rel_doc, expanded_query_terms, mu, tpd, tnd):

    analyzer = EnglishAnalyzer()
    similarity = BM25Similarity(k1,b)
    expanded_q = []

    i = 0
    for qid, q in tqdm(query_all.items(), colour='red', desc='Expanding Queries', leave=False):
    # for qid, q in query_all.items():
     
        i += 1 
        escaped_q = QueryParser('CONTENTS', analyzer).escape(q)      # a few titles had '/' in them which 
        query = QueryParser('CONTENTS', analyzer).parse(escaped_q)
        
        query_terms = [term.strip()[9:] for term in query.toString().split()]
        parsed_q = ' '.join(query_terms)
#         print(parsed_q)
        expension_term_set, docids, ndocid = search(indexReader, query, similarity, top_rel_doc, qid, tpd, tnd)
        # expension_term_set, docids, ndocid = search(indexReader, q, similarity, top_rel_doc, qid, tpd, tnd)
        weights = RM3_term_selection(parsed_q, expension_term_set, docids, ndocid, indexReader, alpha, mu, expanded_query_terms)
        print(qid, q, weights)
        # print(weights.keys())    
        booleanQuery = BooleanQuery.Builder()
        for m, n in weights.items():
            t = Term('CONTENTS', m)
            tq = TermQuery(t)
            boostedTermQuery = BoostQuery(tq, float(n))
            BooleanQuery.setMaxClauseCount(4096)
            booleanQuery.add(boostedTermQuery, BooleanClause.Occur.SHOULD)
        booleanQuery = booleanQuery.build()
       
        expanded_q.append(booleanQuery)   

    return expanded_q

In [11]:
def search_retrived(indexReader, Query, Qid, similarity, out_name):

    searcher = IndexSearcher(indexReader)
    searcher.setSimilarity(similarity)
   
    scoreDocs = searcher.search(Query, 1000).scoreDocs             #retrieving top 1000 relDoc
    i = 1
    res = ''

    for scoreDoc in scoreDocs:
        doc = searcher.doc(scoreDoc.doc)
        r = str(Qid) + '\t' + 'Q0' + '\t' + str(doc.get('ID')) + '\t' + str(i) + '\t' + str(scoreDoc.score) + '\t' + str(out_name) + '\n'
        res += r
        i = i+1   

    return res

In [12]:
def run_RM3(top_PRD, expanded_query_terms, alpha, lamb, tpd, tnd):
    expand_q = expanded_query_BM25(search, RM3_term_selection, k1, b, alpha, top_PRD, expanded_query_terms, lamb, tpd, tnd)
                                       
    name = 'prm_'
    sim = BM25Similarity(k1,b)
    name = name + 'BM25_' + str(k1) + '_'+ str(b)

    file_name = f'./res_TRF/{q_name}/{tpd}_{tnd}_{q_name}_lamb_' + str(lamb) +'_docs_' + str(top_PRD) + '_terms_' + str(expanded_query_terms) + '_alpha_' + str(alpha) +'_tf-idf_' +'.txt'
    out_file = open(file_name, "w")

    res = ''
    for i in tqdm(range(len(query_all)),colour='cyan', desc = 'Re-retrival', leave=False):
    # for i in range(len(query_all)):
    
        result =  search_retrived(indexReader, expand_q[i], list(query_all.keys())[i], sim, name)
        res = res + result

    out_file.write(res)
    out_file.close()
    # print("Retrieval Completed - result dumped in", file_name)

In [13]:
k1 = 0.8
b = 0.4

tpd = [20]
tnd = [20]
top_PRD = [1000]
expanded_query_terms = [50]
alpha = [0.8]
lamb = [0.7]

parameters = list(itertools.product(top_PRD, expanded_query_terms, alpha, lamb, tpd, tnd))

for num_doc, num_q, alpha, lamb, tpd, tnd in tqdm(parameters, colour='red'):
    run_RM3(num_doc, num_q, alpha, lamb, tpd, tnd)

  0%|          | 0/1 [00:00<?, ?it/s]

301 International Organized Crime {'organ': 0.06666666666666665, 'crime': 0.06666666666666665, 'intern': 0.06666666666666665, 'mafia': 0.05592167492837333, 'loeschnak': 0.05152673429283222, 'gelbard': 0.02802754629129367, 'interior': 0.026438495964030823, 'smuggl': 0.025805113430879018, 'brecha': 0.024765815542493977, 'austrian': 0.023119721142472956, 'drug': 0.021697838541550527, 'polic': 0.0209575178674324, 'narcot': 0.020830752743172212, 'traffick': 0.02078139168496386, 'yugomafia': 0.020366448402743282, 'vaculik': 0.020023683875873895, 'tsekhanov': 0.019711595983909154, 'syndic': 0.019050842658628057, 'austria': 0.018622784221679324, 'gang': 0.017213723049621512, 'cri': 0.017027550110553583, 'vienna': 0.014798063348140808, 'stoppach': 0.013491622467629208, 'au1803173194': 0.01334912258391593, 'crack': 0.0132354245030438, 'china': 0.012411513135471385, 'dresden': 0.012082424195893952, 'klimov': 0.01200034122875568, 'slave': 0.01189890379157645, 'iturria': 0.01184117420590447, 'cocai

302 Poliomyelitis and Post-Polio {'polio': 0.1919012292308506, 'poliomyel': 0.08259906866062944, 'post': 0.06666666666666665, 'health': 0.047961845252543066, 'erad': 0.04598996593203477, 'children': 0.037168190043153054, 'diseas': 0.0327998870797346, 'case': 0.027381813003429636, 'china': 0.02538085973834981, 'hemispher': 0.022563335461255482, 'cholera': 0.0174818885420677, 'immun': 0.01530939694773336, 'azerbaijan': 0.015145607853991203, 'quadro': 0.013540963465829423, 'tep': 0.013300101495238907, 'paralysi': 0.012304315960395119, '94we0186a': 0.011866628038699818, 'year': 0.011460260360950617, 'campaign': 0.011332458495164788, 'typhoid': 0.011251939226714218, 'program': 0.011223139808629474, 'wirth': 0.010891426297154908, '2000': 0.010798965321646736, 'ciro': 0.010622690960607237, 'epidem': 0.010362796886932544, 'english': 0.010098083342279792, 'america': 0.010016633490763937, 'outbreak': 0.00992558639932881, 'title:epidemiolog': 0.009832586964625165, 'guo': 0.009741241340763169, 'ma

303 Hubble Telescope Achievements {'hubbl': 0.06666666666666665, 'telescop': 0.06666666666666665, 'achiev': 0.06666666666666665, 'univers': 0.04452687006811385, 'quasar': 0.03401052522890246, 'astronom': 0.03168805021105586, 'eandnaafft': 0.03106193074677941, 'cosmologist': 0.02919451298601396, 'saturn': 0.027493204462404566, 'galaxi': 0.026806537530895347, 'helium': 0.02604269838555839, 'vlt': 0.02395164080898643, 'p9661': 0.022605851641444963, 'goldin': 0.02126867529991936, 'squirrel': 0.020169804479642377, '940114': 0.019129333913555514, 'cobe': 0.01849462289393073, 'light': 0.01849250872417313, 'bang': 0.018345330060266046, 'particl': 0.017900803263007817, 'ingersol': 0.01713532999892329, 'cosmolog': 0.0167028414025036, 'repair': 0.015976556408995552, 'distant': 0.01399217755757806, 'billi': 0.013937866461074786, 'earth': 0.013577896570986998, 'observ': 0.013570993354704209, 'burp': 0.013121121380120776, 'dark': 0.011751141695115638, '8.00': 0.011522356355463241, 'burbidg': 0.01147

304 Endangered Species (Mammals) {'blank': 0.16017443810014934, 'mammal': 0.06666666666666665, 'speci': 0.06666666666666665, 'endang': 0.06666666666666665, 'dolphin': 0.028998158957407733, 'ovi': 0.026912005867984135, 'otter': 0.023272225490184116, 'sanctuari': 0.02253463820584812, 'popul': 0.021696105519963576, 'whale': 0.021312408936364796, 'subspeci': 0.020166571563986713, 'gill': 0.01994084215939112, 'vignei': 0.019407652061826392, 'fish': 0.018792666826114635, 'vessel': 0.018512599222536827, 'certif': 0.015577054535348557, 'lujan': 0.015544568288137738, 'gerrodett': 0.014934403202681018, 'squirrel': 0.014817379845312426, '918': 0.014459619590741803, 'saiga': 0.013628696391465488, 'net': 0.013503547132747942, 'tuna': 0.01330507509868465, 'steller': 0.013126899967336762, 'petrovich': 0.011572144249830071, 'recoveri': 0.01136175106710204, 'florida': 0.01131597084160269, 'grai': 0.011105677111474276, 'fishermen': 0.011049281819374021, 'owner': 0.010694471133487976, 'humpback': 0.01030

...

697 air traffic controller {'faa': 0.07288727459740682, 'air': 0.06666666666666665, 'control': 0.06666666666666665, 'traffic': 0.06666666666666665, 'tracon': 0.0393769484941795, 'hyph': 0.032194484610078185, 'tca': 0.031471222203989443, 'atpac': 0.028471295835052837, 'furlough': 0.027237751717133652, 'airport': 0.025097575355258313, 'wayn': 0.02420721323692464, 'deviat': 0.022963880249594067, 'feder': 0.01929573163079489, 'atct': 0.017979059808715902, 'nprm': 0.017350865435030904, 'hdr': 0.016844330958216365, 'procedur': 0.015947818803923385, 'advisori': 0.01577320195588467, 'clearanc': 0.014861498596368213, '20591': 0.014434570075801954, 'propos': 0.014249315066436164, 'eap': 0.013971761976082096, 'dc': 0.013540443864039374, 'meet': 0.012524335487052979, 'notic': 0.012520736430959918, 'employe': 0.012489212007113136, 'counti': 0.0123444011593042, 'committe': 0.012121356923259195, 'washington': 0.011787493100799032, 'schwartz': 0.011516605313820599, 'altitud': 0.011338165844986302, 'ra

698 literacy rates Africa {'africa': 0.07643946073719973, 'literaci': 0.06666666666666665, 'rate': 0.06666666666666665, 'tunisian': 0.05811293325767103, 'tunisia': 0.03860806462917649, 'hdi': 0.036884010522363286, 'saharan': 0.033997784205928486, 'educ': 0.028930080022548595, 'develop': 0.02867141443353466, 'unesco': 0.0278681772208057, 'girl': 0.023857796094325718, 'build': 0.023772753774049086, 'eltaief': 0.018363134883991036, 'sub': 0.018123318274579775, 'human': 0.01565699047697915, 'bangladesh': 0.015646574710352218, 'engin': 0.014832848343765062, 'compani': 0.014579169079141172, 'polytechniqu': 0.014534750568393554, 'civil': 0.013903250143715637, '677': 0.01374499855040946, 'bredero': 0.0133358850724906, 'global': 0.013142021371527294, 'construct': 0.013114491545030957, 'indigen': 0.013087084461087162, 'spender': 0.01306122056429546, '919': 0.01266494572453025, 'poorer': 0.01254610677548896, 'world': 0.012454993218247272, 'aid': 0.012406412061159581, 'nation': 0.01214500640399241

699 term limits {'term': 0.10955577814782702, 'limit': 0.09999999999999998, 'incumb': 0.06609450965493789, 'officehold': 0.03752180974085435, 'legisl': 0.03172366296422196, 'cawdrei': 0.030041934577046688, 'legislatur': 0.028445638658967255, 'quayl': 0.027112624889569992, 'oklahoma': 0.02441880616556325, 'offic': 0.023611494640642612, 'voter': 0.022532639769493667, 'brown': 0.02108201650519688, 'congress': 0.019169449663887787, 'teglia': 0.01863589022784076, 'lobbyist': 0.017589352982252175, 'reelect': 0.017226381906172226, 'elect': 0.017092438330491087, 'polit': 0.016619318318718868, 'proposit': 0.015707157560932532, 'tenur': 0.015015203008112727, 'opinion': 0.015000430707553362, 'harrar': 0.014982884246439183, 'picu': 0.014558494463328792, '131': 0.014188394350002465, 'lawmak': 0.013835894232083798, 'initi': 0.012952473104671939, 'charter': 0.012780107828220314, 'doerr': 0.012775466494231547, 'california': 0.012336153421860797, 'polsbi': 0.012266063111078003, 'rascal': 0.012198117894

700 gasoline tax U.S. {'tax': 0.13576179455153575, 'u.': 0.06666666666666665, 'gasolin': 0.06666666666666665, 'proposit': 0.03810919932205802, 'ethanol': 0.03558346317254854, 'rfg': 0.035213739263231875, 'bush': 0.03130894577589146, 'billion': 0.026645659920325294, 'poll': 0.024669751863023684, 'california': 0.024036157741462793, 'panetta': 0.021869669135752633, 'voter': 0.020883671462084006, 'deficit': 0.019242992912976726, 'spend': 0.0185564166146402, '111': 0.016747007968732727, 'incom': 0.015614996497690681, 'democrat': 0.0151816549909728, 'etb': 0.015172318181413442, 'oxygen': 0.014991958933073445, 'measur': 0.01472428041470095, 'revolt': 0.014378025457704481, 'transport': 0.013848667050822697, 'republican': 0.013617573929061239, 'deduct': 0.013425635585857852, 'baldassar': 0.013100693942362263, 'deukmejian': 0.012779376853377246, 'mtbe': 0.012518241721968132, 'budget': 0.011714034486204655, 'rep': 0.011512900352196665, '212613': 0.011243186638467748, 'sale': 0.01088979922955671, 

100%|██████████| 1/1 [04:00<00:00, 240.26s/it]
